# Algorithmic Trading Strategy Challenge

Name: Justin Chen

In this challenge, your task is to design an algorithmic trading strategy using the stocks in the provided portfolio. <br>
You may use yfinance to source data for the stocks within the period defined in the dummy_data.xlsx file. <br>
You are not allowed to use a simple "buy and hold" strategy. Be creative and implement a more complex strategy that will maximize profit and minimize risk. <br>
Your solution can be based on machine learning, technical analysis, or any other approach you find suitable. <br>
Be creative and demonstrate your coding ability in this challenge. <br>
In addition to implementing your strategy during this time period, you will also need to graph the profit/loss and liquidity ratio of the strategy over time. <br>
Feel free to add code blocks and markdown blocks as needed, however, ensure specified logic is contained within their respective headers in the notebook.


## Strategy Implementation

### Description of Strategy Implementation
Last semester, I created an algorithmic trading strategy by combining the first/second derivatives of the ONV + 1st/second derivates of the 30 day moving average (to determine buy/sell), and Bollinger Band values to determine quantity to buy/sell (measure of risk), to perform trade on a specific stock. <br />
<br />
So this year, we're going to see if we can have a multi-linear regression model do the job for us instead of trying to code a strategy using technical analysis. <br />
<br />
We're also only going to be buying/selling because the SSMIF overlords will hunt me down if I mention the word shorting within 500 miles of Stevens

### Strategy Steps:
1. Gather all available stocks we have in dummy_data portfolio
2. Gather the timeframe of stocks we have in dummy_data portfolio, and fetch data of stocks 5 years before that date
3. Create a class object to hold all technical indicators we're going to put into the multi-linear regression model
4. Calculate technical indicators (1st d/dx of OBV, 2nd d/dx of OBV, MACD, 30 day moving average, d/dx of that, moving average of past week, d/dx of moving average of past week, Bollinger Bands, RSI)
5. Calculate rate of change of a stock (what we're trying to eventually predict)
6. Plug into multilinear regression (very risky move here because I can't find any literature that does it on technical indicators and 1st rate of change but fortune favors the bold XD)
7. Use multilinear regression on a given stock in dummy_data
8. Get the rate of change of stock for every stock we have in the portfolio. `Find the absolute value of them, sum it, divide each number by the sum` to determine their relative proportions. Then, divide it up by total asset price.
9. If rate of change is positive, rellocate portfolio such that each stock matches that relative proportion. If it's negative, we sell it all. We can always just buy more on the next trading day.

### Library Imports

### Creating Class

### Calculating Technical Indicators

### Training Multilinear Regression

### Determining Allocation + Trading

## Plot P/L

## Plot Liquidity